# 금연구역 3차 전처리 시도

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

# import tensorflow as tf

import requests
import json


In [2]:
PATH = 'D:/project/데이터 분석 프로젝트/이노포스트/data'
# PATH = 'G:/다른 컴퓨터/My_desktop/project/데이터 분석 프로젝트/이노포스트/data'

df_fail = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_실패(4796).csv', encoding ='cp949')
df_save = pd.read_csv(PATH + '/03_금연구역/금연구역_장병용_전처리_성공(40803).csv', encoding ='cp949')

## 남은 4796개는 무엇이 문제일까

In [3]:
df_fail

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,유성구 장대동225-35,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,유성구 죽동150-11,NaN,NaN,NaN


### 샘플을 추출하여 직접 카카오맵에서 검색 진행


In [28]:
a = df_fail.copy()
a['kakao 주소'].sample(20)

2429        유성구 봉명동 656번지 4호 지상1층 102호 
739                    서구 가수원동 847 외1필지
4011             유성구 지족동 907번지 8호 지상1층 
3000             유성구 원내동 342번지 2호 지상1층 
258           유성구 봉명동 536번지 5호 지상2층 220
111              유성구 노은동 553번지 8호 외 1필지
4435                유성구 계룡로141번길 13-18 
34                        중구 중앙로13번길 17
992                서구 도마2동 560번지 13호 2층
2686             유성구 상대동 479번지 1호 지상1층 
3096           유성구 원신흥동 552번지 10호 지상1층 
4219    유성구 봉명동 651번지 1호 신영 미소랑 도시형생활주택
2248         유성구 봉명동 555번지 2호 지상1층 101호
4118            유성구 지족동 917번지 11호 지상1층 
183           유성구 봉명동 466번지 2호 지상1층 101
2945                 유성구 원내동 321번지 11호 
3745            유성구 전민동 306번지 17호 지상1층 
62                       서구 둔산로 220 402
3580            유성구 전민동 448번지 13호 지상1층 
1683                         동구 가양로 86 
Name: kakao 주소, dtype: object

1. 797 :  125번지 2호를 인식하지 못함. 125-2 일때 인식
2. 2990 : 86번지 15호를 인식하지 못함. 86-15 일때 인식
3. 537 : 중구 대종로 307 이라는 도로명번지가 존재하지 않음. 기입 당시의 오탈자로 보임
4. 4078 : 번지 호 이휴
5. 4272 : . 가 존재.  유성구 봉명로 16는 인식함
6. 1262 : 60-8호 라는 형태 인식 못함. 60-8 일때 인식

공통적으로 `***번지` `**호` 형태로 입력된 주소들은 인식하지 못함.   
주소 중간에 `.`가 발견됨 전처리 과정에서 놓친듯

### ***번지 **호 형태의 주소 처리

In [24]:
a[a['주소'].str.contains('번지')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
88,유성구,음식점,디에떼(갑동점),갑동 395번지 1호 지상1,"대전광역시 유성구 갑동 395번지 1호 지상1,2층",2층,유성구 갑동 395번지 1호 지상1,NaN,NaN,NaN
89,유성구,음식점,투투,갑동 394번지 14호 지상1,"대전광역시 유성구 갑동 394번지 14호 지상1, 2층",2층,유성구 갑동 394번지 14호 지상1,NaN,NaN,NaN
90,유성구,음식점,온조갈비,갑동 433번지 23호 지상2,"대전광역시 유성구 갑동 433번지 23호 지상2,3층",3층,유성구 갑동 433번지 23호 지상2,NaN,NaN,NaN
91,유성구,음식점,명품족발칼국수,계산동 739번지 5호 유 플러스,"대전광역시 유성구 계산동 739번지 5호 유 플러스(U+), 지상1층",지상1층,유성구 계산동 739번지 5호 유 플러스,NaN,NaN,NaN
92,유성구,음식점,카페루리,계산동 712번지 2호 지상1층,"대전광역시 유성구 계산동 712번지 2호 지상1층, 효자빌",효자빌,유성구 계산동 712번지 2호 지상1층,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4773,유성구,목욕장,송강목욕탕,송강동 49번지 9호 지상2층,대전광역시 유성구 송강동 49번지 9호 지상2층,NaN,유성구 송강동 49번지 9호 지상2층,NaN,NaN,NaN
4774,유성구,목욕장,구봉파크목욕탕,원내동 228번지 4호 5.10,대전광역시 유성구 원내동 228번지 4호 5.10,NaN,유성구 원내동 228번지 4호 5.10,NaN,NaN,NaN
4775,유성구,목욕장,뉴서울원탕,장대동 286번지 11호,대전광역시 유성구 장대동 286번지 11호,NaN,유성구 장대동 286번지 11호,NaN,NaN,NaN
4776,유성구,목욕장,정원사우나,장대동 331번지 8호,대전광역시 유성구 장대동 331번지 8호,NaN,유성구 장대동 331번지 8호,NaN,NaN,NaN


In [25]:
a[a['주소'].str.contains('호')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12호,NaN,NaN,NaN
54,대덕구,어린이운송용승합차,새봄어린이집,흥룡로 40 210호 가양동,"대전시 동구 흥룡로 40 210호 가양동, 한상아파트",한상아파트,대덕구 흥룡로 40 210호 가양동,NaN,NaN,NaN
88,유성구,음식점,디에떼(갑동점),갑동 395번지 1호 지상1,"대전광역시 유성구 갑동 395번지 1호 지상1,2층",2층,유성구 갑동 395번지 1호 지상1,NaN,NaN,NaN
89,유성구,음식점,투투,갑동 394번지 14호 지상1,"대전광역시 유성구 갑동 394번지 14호 지상1, 2층",2층,유성구 갑동 394번지 14호 지상1,NaN,NaN,NaN
90,유성구,음식점,온조갈비,갑동 433번지 23호 지상2,"대전광역시 유성구 갑동 433번지 23호 지상2,3층",3층,유성구 갑동 433번지 23호 지상2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4773,유성구,목욕장,송강목욕탕,송강동 49번지 9호 지상2층,대전광역시 유성구 송강동 49번지 9호 지상2층,NaN,유성구 송강동 49번지 9호 지상2층,NaN,NaN,NaN
4774,유성구,목욕장,구봉파크목욕탕,원내동 228번지 4호 5.10,대전광역시 유성구 원내동 228번지 4호 5.10,NaN,유성구 원내동 228번지 4호 5.10,NaN,NaN,NaN
4775,유성구,목욕장,뉴서울원탕,장대동 286번지 11호,대전광역시 유성구 장대동 286번지 11호,NaN,유성구 장대동 286번지 11호,NaN,NaN,NaN
4776,유성구,목욕장,정원사우나,장대동 331번지 8호,대전광역시 유성구 장대동 331번지 8호,NaN,유성구 장대동 331번지 8호,NaN,NaN,NaN


In [37]:
df_clean = a.copy()

In [38]:
df_clean["kakao 주소"] = df_clean["kakao 주소"].str.replace(pat='번지', repl=r'', regex=True)
df_clean[df_clean['kakao 주소'].str.contains('번지')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도


In [39]:
df_clean["kakao 주소"] = df_clean["kakao 주소"].str.replace(pat='호', repl=r'', regex=True)
df_clean[df_clean['kakao 주소'].str.contains('호')]

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도


In [40]:
df_clean

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,유성구 장대동225-35,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,유성구 죽동150-11,NaN,NaN,NaN


### api 호출 진행

In [34]:
# rest키 발급 https://developers.kakao.com/console/app

# 병용 474aa3bdba9bd34f1560a7812fafeb9b
# 재인 08d64ac0652c8f9e1c4ea45118910c53
# 수영 616734aa8214dbf958cfaebd6bd41aec
# 태희 04e7e5d98a2c026956de6095963f28f8

rest_api_key = "04e7e5d98a2c026956de6095963f28f8"

In [35]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [36]:
kakao = KakaoLocalAPI(rest_api_key)

In [41]:
add_list = df_clean['kakao 주소']
add_list[0]

'중구 목중로26번길 '

In [42]:
# api 호출 확인
kakao.search_address(add_list[0]) 

{'documents': [{'address': None,
   'address_name': '대전 중구 목중로26번길',
   'address_type': 'ROAD',
   'road_address': {'address_name': '대전 중구 목중로26번길',
    'building_name': '',
    'main_building_no': '',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.412961781981',
    'y': '36.3375130575055',
    'zone_no': ''},
   'x': '127.412961781981',
   'y': '36.3375130575055'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [43]:
for i in tqdm(range(len(add_list))):
    try :
        df_clean['동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_name']
        df_clean['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_clean['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass

print(df_clean.isnull().sum())

  2%|▏         | 88/4796 [00:05<04:25, 17.72it/s]C:\Users\JBY\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 4796/4796 [10:38<00:00,  7.51it/s]

구              0
구분             0
시설명           29
주소             0
번지주소           0
세부주소        4299
kakao 주소       0
동           1182
위도          1182
경도          1182
dtype: int64


In [44]:
df_4th_fail = df_clean[df_clean['위도'].isnull()]
df_4th_save = df_clean[~df_clean['위도'].isnull()]

print(df_4th_fail.shape)
print(df_4th_save.shape)

(1182, 10)
(3614, 10)


`번지`와 `호` 삭제 후 3614개 추가로 인식 성공

데이터프레임 저장

In [90]:
# 병합
df_save2= pd.concat([df_4th_save, df_save], 
                    axis=0,
                    join='outer'
                    )


df_save2.reset_index(drop=True)

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,유성구,음식점,디에떼(갑동점),갑동 395번지 1호 지상1,"대전광역시 유성구 갑동 395번지 1호 지상1,2층",2층,유성구 갑동 395 1 지상1,갑동,36.3606661228153,127.289690866235
1,유성구,음식점,투투,갑동 394번지 14호 지상1,"대전광역시 유성구 갑동 394번지 14호 지상1, 2층",2층,유성구 갑동 394 14 지상1,갑동,36.360363448458,127.292967775578
2,유성구,음식점,온조갈비,갑동 433번지 23호 지상2,"대전광역시 유성구 갑동 433번지 23호 지상2,3층",3층,유성구 갑동 433 23 지상2,갑동,36.3604408803087,127.295411577756
3,유성구,음식점,명품족발칼국수,계산동 739번지 5호 유 플러스,"대전광역시 유성구 계산동 739번지 5호 유 플러스(U+), 지상1층",지상1층,유성구 계산동 739 5 유 플러스,계산동,36.3404445142985,127.303949806132
4,유성구,음식점,카페루리,계산동 712번지 2호 지상1층,"대전광역시 유성구 계산동 712번지 2호 지상1층, 효자빌",효자빌,유성구 계산동 712 2 지상1층,계산동,36.3442073881703,127.298405817194
...,...,...,...,...,...,...,...,...,...,...
44412,동구,음식점,에꿍이치킨,충무로 244,대전광역시 동구 충무로 244 (신흥동),NaN,동구 충무로 244,신흥동,36.322869,127.442877
44413,동구,음식점,종가소머리곰탕,우암로 241,대전광역시 동구 우암로 241 (가양동),NaN,동구 우암로 241,가양동,36.344371,127.44301
44414,동구,음식점,다해어죽 가양점,가양로 142-3,대전광역시 동구 가양로 142-3 (가양동),NaN,동구 가양로 142-3,가양동,36.349017,127.441479
44415,동구,음식점,띵호와식당,동서대로 1702,대전광역시 동구 동서대로 1702 (가양동),NaN,동구 동서대로 1702,가양동,36.349428,127.438749


In [96]:
df_save2.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(44417).csv", index=False, encoding='cp949')

## 남은 1182개는 무엇이 문제일까

In [45]:
df_4th_fail

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,유성구 장대동225-35,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,유성구 죽동150-11,NaN,NaN,NaN


### 주소 + 시설명을 조합하여 검색 시도

In [58]:
df_build = df_4th_fail.copy()

In [59]:
df_build['kakao 주소'] = df_build['kakao 주소'].str.cat(df_build['시설명'], sep=' ')

In [60]:
df_build

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길 중앙독서실,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46 두드림음악학원,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12 은혜어린이집,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8 꿈나무어린이집,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0 정보통신학교 교육생 숙소,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,NaN,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4 기동화력 정비공장,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,NaN,NaN,NaN,NaN


중복된 공백 제거

In [62]:
print(df_build['kakao 주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_build['kakao 주소'].str.contains('   ').sum()) # 스페이스바 세번
print(df_build['kakao 주소'].str.contains('    ').sum()) # 스페이스바 네번

699
5
0


In [64]:
# 전부 0이 될때까지 여러번 실행
df_build["kakao 주소"] = df_build["kakao 주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_build['kakao 주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_build['kakao 주소'].str.contains('   ').sum()) # 스페이스바 세번

0
0


In [66]:
build_list = df_build['kakao 주소']
build_list[1]

'중구 계백로1571번길 46 두드림음악학원'

In [65]:
# api 호출 확인
kakao.search_address(build_list[0]) 

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [67]:
for i in tqdm(range(len(build_list))):
    try :
        df_build['동'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['address']['region_3depth_name']
        df_build['경도'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['x']
        df_build['위도'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['y']
    except:
        pass

print(df_build.isnull().sum())

100%|██████████| 1182/1182 [00:33<00:00, 35.14it/s] 

구              0
구분             0
시설명           29
주소             0
번지주소           0
세부주소        1011
kakao 주소      29
동           1142
위도          1142
경도          1142
dtype: int64


In [68]:
df_5th_fail = df_build[df_build['위도'].isnull()]
df_5th_save = df_build[~df_build['위도'].isnull()]

print(df_5th_fail.shape)
print(df_5th_save.shape)

(1142, 10)
(40, 10)


40개 추가로 인식 성공

### 데이터 프레임 저장

In [93]:
# 병합
df_save3= pd.concat([df_5th_save, df_save2], 
                    axis=0,
                    join='outer'
                    )


df_save3.reset_index(drop=True)
df_save3.shape

(44457, 10)

In [95]:
df_save3.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(44457).csv", index=False, encoding='cp949')

## 실패한 주소들 다시 확인

In [69]:
df_5th_fail

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,"대전광역시 중구 목중로26번길 , 83 (중촌동, 현대아파트)",83,중구 목중로26번길 중앙독서실,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,"대전광역시 중구 계백로1571번길 46 , 2층 (유천동)",2층,중구 계백로1571번길 46 두드림음악학원,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,"대전 중구 보문로 207번길14 문화어린이집, 문화유치원",문화유치원,중구 보문로 207번길14 문화어린이집 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,"대전 중구 태평동 삼부@ 410동 12호, 은혜어린이집",은혜어린이집,중구 태평동 삼부아파트 410동 12 은혜어린이집,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,"대전 중구 선화로57번길 8, 4층(선화동) 꿈나무어린이집",4층 꿈나무어린이집,중구 선화로57번길 8 꿈나무어린이집,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0 정보통신학교 교육생 숙소,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,NaN,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4 기동화력 정비공장,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,NaN,NaN,NaN,NaN


번지주소에서 다시 `,` 삭제 후 시도

In [72]:
df_5th_fail["번지주소"] = df_5th_fail["번지주소"].str.replace(pat=',', repl=r'', regex=True)
df_5th_fail[df_5th_fail['번지주소'].str.contains(',')]

c:\Users\JBY\anaconda3\envs\3.7\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도


In [73]:
df_5th_fail

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,대전광역시 중구 목중로26번길 83 (중촌동 현대아파트),83,중구 목중로26번길 중앙독서실,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,대전광역시 중구 계백로1571번길 46 2층 (유천동),2층,중구 계백로1571번길 46 두드림음악학원,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,대전 중구 보문로 207번길14 문화어린이집 문화유치원,문화유치원,중구 보문로 207번길14 문화어린이집 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,대전 중구 태평동 삼부@ 410동 12호 은혜어린이집,은혜어린이집,중구 태평동 삼부아파트 410동 12 은혜어린이집,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,대전 중구 선화로57번길 8 4층(선화동) 꿈나무어린이집,4층 꿈나무어린이집,중구 선화로57번길 8 꿈나무어린이집,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0 정보통신학교 교육생 숙소,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,NaN,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4 기동화력 정비공장,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,NaN,NaN,NaN,NaN


In [79]:
import re

regex = "\(.*\)|\s-\s.*"

for i in range(len(df_5th_fail['번지주소'])):
    df_5th_fail['번지주소'][i] = re.sub(regex, '', df_5th_fail['번지주소'][i])
    print(df_5th_fail['번지주소'])

0                  대전광역시 중구 목중로26번길   83 
1              대전광역시 중구 계백로1571번길 46  2층 
2       대전 중구 보문로 207번길14   문화어린이집  문화유치원
3           대전 중구 태평동 삼부@ 410동 12호 은혜어린이집
4              대전 중구 선화로57번길 8 4층 꿈나무어린이집
                      ...                
4791                   대전광역시 유성구 자운동223-0
4792                  대전광역시 유성구 장대동225-35
4793                    대전광역시 유성구 장동267-4
4794                   대전광역시 유성구 죽동150-11
4795                   대전광역시 유성구 추목동560-0
Name: 번지주소, Length: 1142, dtype: object
0                  대전광역시 중구 목중로26번길   83 
1              대전광역시 중구 계백로1571번길 46  2층 
2       대전 중구 보문로 207번길14   문화어린이집  문화유치원
3           대전 중구 태평동 삼부@ 410동 12호 은혜어린이집
4              대전 중구 선화로57번길 8 4층 꿈나무어린이집
                      ...                
4791                   대전광역시 유성구 자운동223-0
4792                  대전광역시 유성구 장대동225-35
4793                    대전광역시 유성구 장동267-4
4794                   대전광역시 유성구 죽동150-11
4795                   대전광역시 유성구 추목동560-0
Name: 번지주소, Length: 1142, dtype: obj

c:\Users\JBY\anaconda3\envs\3.7\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyError: 5

KeyError가 발생했지만 괄호제거는 성공

In [80]:
df_5th_fail

,구,구분,시설명,주소,번지주소,세부주소,kakao 주소,동,위도,경도
0,중구,학교교과교습학원,중앙독서실,목중로26번길,대전광역시 중구 목중로26번길 83,83,중구 목중로26번길 중앙독서실,NaN,NaN,NaN
1,중구,학교교과교습학원,두드림음악학원,계백로1571번길 46,대전광역시 중구 계백로1571번길 46 2층,2층,중구 계백로1571번길 46 두드림음악학원,NaN,NaN,NaN
2,중구,어린이운송용 승합자동차,문화어린이집,보문로 207번길14 문화어린이집,대전 중구 보문로 207번길14 문화어린이집 문화유치원,문화유치원,중구 보문로 207번길14 문화어린이집 문화어린이집,NaN,NaN,NaN
3,중구,어린이운송용 승합자동차,은혜어린이집,태평동 삼부아파트 410동 12호,대전 중구 태평동 삼부@ 410동 12호 은혜어린이집,은혜어린이집,중구 태평동 삼부아파트 410동 12 은혜어린이집,NaN,NaN,NaN
4,중구,어린이운송용 승합자동차,꿈나무어린이집,선화로57번길 8,대전 중구 선화로57번길 8 4층 꿈나무어린이집,4층 꿈나무어린이집,중구 선화로57번길 8 꿈나무어린이집,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4791,유성구,복합용도,정보통신학교 교육생 숙소,자운동223-0,대전광역시 유성구 자운동223-0,NaN,유성구 자운동223-0 정보통신학교 교육생 숙소,NaN,NaN,NaN
4792,유성구,복합용도,NaN,장대동225-35,대전광역시 유성구 장대동225-35,NaN,NaN,NaN,NaN,NaN
4793,유성구,복합용도,기동화력 정비공장,장동267-4,대전광역시 유성구 장동267-4,NaN,유성구 장동267-4 기동화력 정비공장,NaN,NaN,NaN
4794,유성구,복합용도,NaN,죽동150-11,대전광역시 유성구 죽동150-11,NaN,NaN,NaN,NaN,NaN


In [84]:
# 전부 0이 될때까지 여러번 실행
df_5th_fail["번지주소"] = df_5th_fail["번지주소"].str.replace(pat='  ', repl=r' ', regex=True)
print(df_5th_fail['번지주소'].str.contains('  ').sum()) # 스페이스바 두번
print(df_5th_fail['번지주소'].str.contains('   ').sum()) # 스페이스바 세번

0
0


c:\Users\JBY\anaconda3\envs\3.7\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [85]:
df_fail = df_5th_fail.copy()

In [86]:
build_list = df_fail['번지주소']
build_list[1]

'대전광역시 중구 계백로1571번길 46 2층 '

In [87]:
# api 호출 확인
kakao.search_address(build_list[0]) 

{'documents': [{'address': {'address_name': '대전 중구 중촌동 116-8',
    'b_code': '3014010400',
    'h_code': '3014056000',
    'main_address_no': '116',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_h_name': '중촌동',
    'region_3depth_name': '중촌동',
    'sub_address_no': '8',
    'x': '127.414436051345',
    'y': '36.3387070586325'},
   'address_name': '대전 중구 목중로26번길 83',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '대전 중구 목중로26번길 83',
    'building_name': '',
    'main_building_no': '83',
    'region_1depth_name': '대전',
    'region_2depth_name': '중구',
    'region_3depth_name': '중촌동',
    'road_name': '목중로26번길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.414436051345',
    'y': '36.3387070586325',
    'zone_no': '34804'},
   'x': '127.414436051345',
   'y': '36.3387070586325'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [88]:
for i in tqdm(range(len(build_list))):
    try :
        df_fail['동'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['address']['region_3depth_name']
        df_fail['경도'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['x']
        df_fail['위도'].iloc[i] = kakao.search_address(build_list[i])['documents'][0]['y']
    except:
        pass

print(df_fail.isnull().sum())

100%|██████████| 1142/1142 [00:27<00:00, 41.24it/s] 

구              0
구분             0
시설명           28
주소             0
번지주소           0
세부주소         985
kakao 주소      28
동           1124
위도          1124
경도          1124
dtype: int64


In [89]:
df_6th_fail = df_fail[df_fail['위도'].isnull()]
df_6th_save = df_fail[~df_fail['위도'].isnull()]

print(df_6th_fail.shape)
print(df_6th_save.shape)

(1124, 10)
(18, 10)


18개 추가로 인식 성공

나머지 주소들은 직접 카카오맵에 검색해봤을 때 대체로 없는 주소로 뜨는 것을 확인함


### 데이터프레임 저장

In [97]:
# 병합
df_save4= pd.concat([df_6th_save, df_save3], 
                    axis=0,
                    join='outer'
                    )


df_save4.reset_index(drop=True)
df_save4.shape

(44475, 10)

In [98]:
df_save4.to_csv(PATH + "/03_금연구역/금연구역_장병용_전처리_성공(44475).csv", index=False, encoding='cp949')